In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## 1. Read the .xlsx file:

#### a. Assign the result to a variable named “data”

In [2]:
data=pd.read_excel('/Users/yubinye/Downloads/BRtest_Yubin/Data_eShop.xlsx')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464433 entries, 0 to 464432
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   itemid              464433 non-null  int64         
 1   shopid              464433 non-null  int64         
 2   item_name           464409 non-null  object        
 3   item_description    463336 non-null  object        
 4   item_variation      464433 non-null  object        
 5   price               464433 non-null  float64       
 6   stock               464433 non-null  int64         
 7   category            464422 non-null  object        
 8   cb_option           464433 non-null  int64         
 9   is_preferred        464433 non-null  int64         
 10  sold_count          464433 non-null  int64         
 11  item_creation_date  464433 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(6), object(4)
memory usage: 42.5+ MB


In [4]:
data.head(3)

,itemid,shopid,item_name,item_description,item_variation,price,stock,category,cb_option,is_preferred,sold_count,item_creation_date
0,1925574,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355333 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:24:42
1,1925617,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355888 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:26:37
2,943600,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-25 21:02:13


#### b. Assign all column names to a variable named “columns”

In [5]:
columns = data.columns
print(columns)

Index(['itemid', 'shopid', 'item_name', 'item_description', 'item_variation',
       'price', 'stock', 'category', 'cb_option', 'is_preferred', 'sold_count',
       'item_creation_date'],
      dtype='object')


#### c. Assign a new column named “price_level” with values that follow the rules:

 - 0 - price less than 100 
 - 1 - price between 100 and 200 
 - 2- price greater than 200

In [6]:
# find the max value of price
data['price'].max()+1

1000000000.0

In [7]:
# set bins
bins=[0,100,200,1000000000]

In [8]:
data['price_level'] = pd.cut(data['price'], bins)

In [9]:
data['price_level'].value_counts()

(0, 100]             444334
(100, 200]            10887
(200, 1000000000]      9212
Name: price_level, dtype: int64

## 2. Find:

#### a. How many unique shops are in the dataset?

In [10]:
len(pd.unique(data['shopid']))

7856

#### b. How many unique preferred and cross border shops are in the dataset?

In [11]:
prefer = data.query('is_preferred == 1')

In [12]:
len(pd.unique(prefer['shopid']))

533

In [13]:
cross = data.query('cb_option == 1')

In [14]:
len(pd.unique(cross['shopid']))

2557

In [32]:
prefer_cross = data.query('is_preferred == 1 & cb_option == 1 ')
len(pd.unique(prefer_cross['shopid']))

158

#### c. How many products have zero sold count?

In [15]:
sold_0 = data.query('sold_count == 0')

In [16]:
len(pd.unique(sold_0['itemid']))

438092

#### d. How many products were created in the year 2018?

In [17]:
after_2018 = data.query('item_creation_date > 2018')

In [18]:
len(pd.unique(after_2018['itemid']))

177407

## 3. Find:

#### a. Top 3 Preferred shops’ shopid that have the largest number of unique products 

In [19]:
prefer.groupby(by=["shopid"])['itemid'].count().sort_values(ascending=False)[0:3]

shopid
43384791    2034
6072241     1998
26974701    1037
Name: itemid, dtype: int64

#### b. Top 3 Categories that have the largest number of unique cross-border productsm

In [20]:
cross.groupby(by=["category"])['itemid'].count().sort_values(ascending=False)[0:3]

category
Women's Apparel     80494
Mobile & Gadgets    58760
Men's Wear          51795
Name: itemid, dtype: int64

## 4. Find Top 3 shopid 

#### with the highest revenue (Assumption: the product price has not been changed.)

In [21]:
data['revenue'] = data['price']*data['sold_count']

In [34]:
data.groupby(by=["shopid"])['revenue'].sum().sort_values(ascending=False)[0:3]

shopid
18796234    177897.79
30769353     63491.39
30628794     39852.00
Name: revenue, dtype: float64

## 5. Find number of products 
#### that have more than 3 variations (do not include products with 3 or fewer variations)

In [23]:
num = []
for i in data.index:
    var_str = data['item_variation'][i]
    counter = var_str.count(':')
    num.append(counter)

In [24]:
data['num_variations'] = num

In [25]:
len(data.query('num_variations > 3')['itemid'])

242150

## 6. Identify duplicated listings within each shop 

#### (If listing A and B in shop S have the exactly same product title, product detailed description, and price, both listing A and B are considered as duplicated listings)

#### a. Mark those duplicated listings with True otherwise False and store the marking result in a new column named “is_duplicated”

In [26]:
data['is_duplicated'] = False
data['is_duplicated'][data[['shopid','item_name', 'item_description', 'price']].duplicated() == True] = True

#### b. Find duplicate listings that has less than 2 sold count and store the result in a new excel file named “duplicated_listings.xlsx”

In [27]:
duplicated_listings = data.query('sold_count < 2 & is_duplicated == True')

In [28]:
duplicated_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192357 entries, 9 to 464424
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   itemid              192357 non-null  int64         
 1   shopid              192357 non-null  int64         
 2   item_name           192357 non-null  object        
 3   item_description    191863 non-null  object        
 4   item_variation      192357 non-null  object        
 5   price               192357 non-null  float64       
 6   stock               192357 non-null  int64         
 7   category            192354 non-null  object        
 8   cb_option           192357 non-null  int64         
 9   is_preferred        192357 non-null  int64         
 10  sold_count          192357 non-null  int64         
 11  item_creation_date  192357 non-null  datetime64[ns]
 12  price_level         192357 non-null  category      
 13  revenue             192357 no

In [29]:
duplicated_listings.to_excel('/Users/yubinye/Downloads/BRtest_Yubin/duplicated_listings.xlsx')

#### c. Find the preferred shop shopid that have the most number of duplicated listings

In [30]:
prefer_dup = data.query('is_preferred == 1 & is_duplicated == True')

In [31]:
prefer_dup.groupby(by=["shopid"])['itemid'].count().sort_values(ascending=False)[0:1]

shopid
43384791    1933
Name: itemid, dtype: int64